
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nuyoahwjl/SISR/blob/main/demo/utils.ipynb)


In [ ]:
from PIL import Image
import os
import json
from math import log10, sqrt
import numpy as np
from skimage.metrics import structural_similarity as ssim

def rgb2y(img: Image.Image) -> np.ndarray:
    y, _, _ = img.convert('YCbCr').split()
    return np.asarray(y).astype('float32')

def psnr_y(img_sr: Image.Image, img_hr: Image.Image, shave=4) -> float:
    y_sr = rgb2y(img_sr)
    y_hr = rgb2y(img_hr)
    if shave > 0:
        y_sr = y_sr[shave:-shave, shave:-shave]
        y_hr = y_hr[shave:-shave, shave:-shave]
    mse = np.mean((y_sr - y_hr) ** 2)
    return 99.0 if mse < 1e-10 else 10 * log10((255.0 ** 2) / mse)

def ssim_y(img_sr: Image.Image, img_hr: Image.Image, shave=4) -> float:
    y_sr = rgb2y(img_sr)
    y_hr = rgb2y(img_hr)
    if shave > 0:
        y_sr = y_sr[shave:-shave, shave:-shave]
        y_hr = y_hr[shave:-shave, shave:-shave]
    return float(ssim(y_hr, y_sr, data_range=255.0, channel_axis=None))

def bicubic(input_dir, scale: int):
    img_paths = sorted(
        [
            os.path.join(input_dir, p)
            for p in os.listdir(input_dir)
            if p.lower().endswith(('.png', 'jpg', 'jpeg', 'bmp'))
        ]
    )
    out_dir_lr = f'lr_bicubic_x{scale}'
    os.makedirs(out_dir_lr, exist_ok=True)
    out_dir_hr = f'hr_bicubic_x{scale}'
    os.makedirs(out_dir_hr, exist_ok=True)

    metrics = []
    num = 0
    psnr_total = 0.0
    ssim_total = 0.0

    for idx, img_path in enumerate(img_paths, start=1):
        num += 1
        img = Image.open(img_path).convert('RGB')
        w, h = img.size
        img = img.resize((round(w / 12) * 12, round(h / 12) * 12), Image.BICUBIC)

        lr_w, lr_h = int(img.width / scale), int(img.height / scale)
        lr = img.resize((lr_w, lr_h), Image.BICUBIC)
        lr.save(os.path.join(out_dir_lr, f'{idx:02d}_LR.png'))

        hr_w, hr_h = int(lr.width * scale), int(lr.height * scale)
        hr = lr.resize((hr_w, hr_h), Image.BICUBIC)
        hr.save(os.path.join(out_dir_hr, f'{idx:02d}_HR.png'))
        psnr_value = psnr_y(img, hr, shave=scale)
        ssim_value = ssim_y(img, hr, shave=scale)
        psnr_total += psnr_value
        ssim_total += ssim_value
        metrics.append({
            "image_index": idx,
            "psnr_y": psnr_value,
            "ssim_y": ssim_value
        })
    
    avg_psnr = psnr_total / num
    avg_ssim = ssim_total / num
    summary = {
        "num_images": num,
        "psnr_y": avg_psnr,
        "ssim_y": avg_ssim,
        "image_metrics": metrics
    }
    metrics_file = os.path.join(out_dir_hr, 'metrics.json')
    with open(metrics_file, 'w') as f:
        json.dump(summary, f, indent=2)


if __name__ == '__main__':
    bicubic('original',scale=2)
    bicubic('original',scale=3)
    bicubic('original',scale=4)
  

In [ ]:
from PIL import Image, ImageDraw

def zoom_inset(
    img: Image.Image,
    rect: tuple,           # 要放大的块(x, y, w, h)
    scale: int = 2,        # 放大倍数
    corner: str = "bottom_right",  # 贴图角落：top_left/top_right/bottom_left/bottom_right
    rect_color=(0, 102, 255),      # 原位置框的颜色
    rect_thickness: int = 4,       # 原位置框线宽
    inset_border_color=(0, 102, 255),  # 角落大图边框颜色
    inset_border: int = 4,         # 角落大图边框线宽
    margin: int = 16,              # 距离边缘的外边距
    connector: bool = True,        # 是否画连线
    resample="bicubic",            # "nearest" / "bilinear"/ "bicubic"
):
    x, y, w, h = rect
    img = img.convert("RGB")
    W, H = img.size

    # 取ROI并放大
    roi = img.crop((x, y, x + w, y + h))
    from PIL import Image as PILImage
    resample_map = {
        "nearest": PILImage.Resampling.NEAREST,
        "bilinear": PILImage.Resampling.BILINEAR,
        "bicubic": PILImage.Resampling.BICUBIC,
    }
    roi_big = roi.resize((w * scale, h * scale), resample=resample_map.get(resample, PILImage.Resampling.BICUBIC))

    # 角落位置
    corners = {
        "top_left": (margin, margin),
        "top_right": (W - margin - roi_big.width, margin),
        "bottom_left": (margin, H - margin - roi_big.height),
        "bottom_right": (W - margin - roi_big.width, H - margin - roi_big.height),
    }
    ix, iy = corners[corner]

    out = img.copy()
    draw = ImageDraw.Draw(out)

    # 原图上画框
    for t in range(rect_thickness):
        draw.rectangle([x - t, y - t, x + w + t, y + h + t], outline=rect_color)

    # 贴放大图
    out.paste(roi_big, (ix, iy))

    # 放大图边框
    for t in range(inset_border):
        draw.rectangle([ix - t, iy - t, ix + roi_big.width + t, iy + roi_big.height + t],
                       outline=inset_border_color)

    # 连线（可选）
    if connector:
        src = {
            "top_left": (x, y),
            "top_right": (x + w, y),
            "bottom_left": (x, y + h),
            "bottom_right": (x + w, y + h),
        }[corner]
        dst = {
            "top_left": (ix + roi_big.width, iy + roi_big.height),
            "top_right": (ix, iy + roi_big.height),
            "bottom_left": (ix + roi_big.width, iy),
            "bottom_right": (ix, iy),
        }[corner]
        draw.line([src, dst], fill=inset_border_color, width=max(2, rect_thickness - 1))

    return out


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img1 = Image.open("original/09.png")
img2 = Image.open("hr_bicubic_x2/09_HR.png")
img3 = Image.open("srcnn_x2/09_SR.png")
img4 = Image.open("fsrcnn_x2/09_SR.png")
img5 = Image.open("espcn_x2/09_SR.png")
img6 = Image.open("edsr_x2/09_SR.png")
img7 = Image.open("imdn_x2/09_SR.png")

params = {
    "rect": (250, 150, 90, 90), 
    "scale": 3,
    "corner": "bottom_right",
    "rect_color": (0, 191, 255), 
    "rect_thickness": 8,
    "inset_border_color": (0, 191, 255), 
    "inset_border": 8,
    "connector": False,
    "margin": 8
}

out1 = zoom_inset(img1, **params)
out2 = zoom_inset(img2, **params)
out3 = zoom_inset(img3, **params)
out4 = zoom_inset(img4, **params)
out5 = zoom_inset(img5, **params)
out6 = zoom_inset(img6, **params)
out7 = zoom_inset(img7, **params)

fig, axes = plt.subplots(1, 7, figsize=(28, 5))
imgs = [out1, out2, out3, out4, out5, out6, out7]
titles = ["Original", "Bicubic x2", "SRCNN x2", "FSRCNN x2", "ESPCN x2", "EDSR x2", "IMDN x2"]
values = [
    "(PSNR/SSIM)",
    "(32.12dB/0.9617)",
    "(35.51dB/0.9656)",
    "(33.70dB/0.9647)",
    "(33.63dB/0.9648)",
    "(37.05dB/0.9776)",
    "(37.16dB/0.9777)"
]

for i, (ax, img, title, val) in enumerate(zip(axes, imgs, titles, values)):
    ax.imshow(img)
    ax.set_title(title, fontsize=16, pad=8)
    ax.text(0.5, -0.05, val, fontsize=15, ha='center', va='top', transform=ax.transAxes)
    ax.axis("off")

fig.suptitle("Zoom-In Insets for x2 Super-Resolution Results(Set14-monarch)", fontsize=20, fontweight='bold', y=0.98)
plt.subplots_adjust(wspace=0.02, left=0.01, right=0.99, top=0.88, bottom=0.18)
plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img1 = Image.open("original/10.png")
img2 = Image.open("hr_bicubic_x3/10_HR.png")
img3 = Image.open("srcnn_x3/10_SR.png")
img4 = Image.open("fsrcnn_x3/10_SR.png")
img5 = Image.open("espcn_x3/10_SR.png")
img6 = Image.open("edsr_x3/10_SR.png")
img7 = Image.open("imdn_x3/10_SR.png")

params = {
    "rect": (100, 60, 60, 60), 
    "scale": 3,
    "corner": "bottom_right",
    "rect_color": (255, 0, 0), 
    "rect_thickness": 5,
    "inset_border_color": (255, 0, 0), 
    "inset_border": 5,
    "connector": False,
    "margin": 6
}

out1 = zoom_inset(img1, **params)
out2 = zoom_inset(img2, **params)
out3 = zoom_inset(img3, **params)
out4 = zoom_inset(img4, **params)
out5 = zoom_inset(img5, **params)
out6 = zoom_inset(img6, **params)
out7 = zoom_inset(img7, **params)

fig, axes = plt.subplots(1, 7, figsize=(28, 5))
imgs = [out1, out2, out3, out4, out5, out6, out7]
titles = ["Original", "Bicubic x3", "SRCNN x3", "FSRCNN x3", "ESPCN x3", "EDSR x3", "IMDN x3"]
values = [
    "(PSNR/SSIM)",
    "(25.74dB/0.8173)",
    "(27.05dB/0.8557)",
    "(27.18dB/0.8593)",
    "(27.01dB/0.8553)",
    "(28.72dB/0.8770)",
    "(28.72dB/0.8762)"
]

for i, (ax, img, title, val) in enumerate(zip(axes, imgs, titles, values)):
    ax.imshow(img)
    ax.set_title(title, fontsize=16, pad=8)
    ax.text(0.5, -0.05, val, fontsize=15, ha='center', va='top', transform=ax.transAxes)
    ax.axis("off")

fig.suptitle("Zoom-In Insets for x3 Super-Resolution Results(Set14-zebra)", fontsize=20, fontweight='bold', y=0.98)
plt.subplots_adjust(wspace=0.02, left=0.01, right=0.99, top=0.88, bottom=0.18)
plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img1 = Image.open("original/08.png")
img2 = Image.open("hr_bicubic_x4/08_HR.png")
img3 = Image.open("srcnn_x4/08_SR.png")
img4 = Image.open("fsrcnn_x4/08_SR.png")
img5 = Image.open("espcn_x4/08_SR.png")
img6 = Image.open("edsr_x4/08_SR.png")
img7 = Image.open("imdn_x4/08_SR.png")

params = {
    "rect": (70, 50, 80, 60), 
    "scale": 3,
    "corner": "bottom_right",
    "rect_color": (50, 205, 50), 
    "rect_thickness": 5,
    "inset_border_color": (50, 205, 50),
    "inset_border": 5,
    "connector": False,
    "margin": 6
}

out1 = zoom_inset(img1, **params)
out2 = zoom_inset(img2, **params)
out3 = zoom_inset(img3, **params)
out4 = zoom_inset(img4, **params)
out5 = zoom_inset(img5, **params)
out6 = zoom_inset(img6, **params)
out7 = zoom_inset(img7, **params)

fig, axes = plt.subplots(1, 7, figsize=(28, 5))
imgs = [out1, out2, out3, out4, out5, out6, out7]
titles = ["Original", "Bicubic x4", "SRCNN x4", "FSRCNN x4", "ESPCN x4", "EDSR x4", "IMDN x4"]
values = [
    "(PSNR/SSIM)",
    "(24.40dB/0.7328)",
    "(24.90dB/0.7581)",
    "(24.94dB/0.7542)",
    "(24.85dB/0.7418)",
    "(25.90dB/0.7503)",
    "(25.71dB/0.7809)"
]

for i, (ax, img, title, val) in enumerate(zip(axes, imgs, titles, values)):
    ax.imshow(img)
    ax.set_title(title, fontsize=16, pad=8)
    ax.text(0.5, -0.05, val, fontsize=15, ha='center', va='top', transform=ax.transAxes)
    ax.axis("off")

fig.suptitle("Zoom-In Insets for x4 Super-Resolution Results(Set14-flowers)", fontsize=20, fontweight='bold', y=0.98)
plt.subplots_adjust(wspace=0.02, left=0.01, right=0.99, top=0.88, bottom=0.18)
plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

rect = (700, 300, 400 ,300)
x, y, w, h = rect

img1 = Image.open("original/04.png")
img1 = zoom_inset(
    img1,
    rect = rect,
    scale = 2,
    corner = "bottom_right",
    rect_color = (255, 255, 0),
    rect_thickness = 15,   
    inset_border_color = (255, 255, 0),
    inset_border = 15,      
    margin = 10,  
    connector = False   
)
img2 = Image.open("hr_bicubic_x2/04_HR.png").convert("RGB").crop((x, y, x + w, y + h))
img3 = Image.open("srcnn_x2/04_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img4 = Image.open("fsrcnn_x2/04_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img5 = Image.open("espcn_x2/04_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img6 = Image.open("edsr_x2/04_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img7 = Image.open("imdn_x2/04_SR.png").convert("RGB").crop((x, y, x + w, y + h))

imgs = [img1, img2, img3, img4, img5, img6, img7]
titles = ["Original", "Bicubic x2", "SRCNN x2", "FSRCNN x2", "ESPCN x2", "EDSR x2", "IMDN x2"]
values = [
    "(PSNR/SSIM)",
    "(36.20dB/0.9712)",
    "(37.32dB/0.9774)",
    "(37.57dB/0.9773)",
    "(37.25dB/0.9751)",
    "(38.41dB/0.9819)",
    "(38.44dB/0.9819)"
]

fig = plt.figure(figsize=(25, 8), constrained_layout=True)
gs = fig.add_gridspec(2, 5) 

ax1 = fig.add_subplot(gs[:, :2])
ax1.imshow(imgs[0])
ax1.set_title(titles[0], fontsize=30, pad=15)
ax1.text(0.5, -0.03, values[0], fontsize=25, ha='center', va='top', transform=ax1.transAxes)
ax1.axis("off")

idx = 0
for r in range(2):
    for c in range(3):
        ax = fig.add_subplot(gs[r, 2 + c])
        ax.imshow(imgs[idx + 1])
        ax.set_title(titles[idx + 1], fontsize=25, pad=8)
        ax.text(0.5, -0.05, values[idx + 1], fontsize=18, ha='center', va='top', transform=ax.transAxes)
        ax.axis("off")
        idx += 1

fig.suptitle("Zoom-In Insets for x2 Super-Resolution Results(DIV2k-0368)", fontsize=30, fontweight='bold', y=1.1)
plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

rect = (1400, 700, 400 ,300)
x, y, w, h = rect

img1 = Image.open("original/03.png")
img1 = zoom_inset(
    img1,
    rect = rect,
    scale = 2,
    corner = "bottom_left",
    rect_color = (255, 0, 255),
    rect_thickness = 15,   
    inset_border_color = (255, 0, 255), 
    inset_border = 15,      
    margin = 10,  
    connector = False   
)
img2 = Image.open("hr_bicubic_x3/03_HR.png").convert("RGB").crop((x, y, x + w, y + h))
img3 = Image.open("srcnn_x3/03_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img4 = Image.open("fsrcnn_x3/03_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img5 = Image.open("espcn_x3/03_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img6 = Image.open("edsr_x3/03_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img7 = Image.open("imdn_x3/03_SR.png").convert("RGB").crop((x, y, x + w, y + h))

imgs = [img1, img2, img3, img4, img5, img6, img7]
titles = ["Original", "Bicubic x3", "SRCNN x3", "FSRCNN x3", "ESPCN x3", "EDSR x3", "IMDN x3"]
values = [
    "(PSNR/SSIM)",
    "(31.74dB/0.9272)",
    "(32.64dB/0.9370)",
    "(32.71dB/0.9317)",
    "(32.41dB/0.9254)",
    "(34.60dB/0.9524)",
    "(34.72dB/0.9533)"
]

fig = plt.figure(figsize=(25, 8), constrained_layout=True)
gs = fig.add_gridspec(2, 5)

ax1 = fig.add_subplot(gs[:, :2])
ax1.imshow(imgs[0])
ax1.set_title(titles[0], fontsize=30, pad=15)
ax1.text(0.5, -0.03, values[0], fontsize=25, ha='center', va='top', transform=ax1.transAxes)
ax1.axis("off")

idx = 0
for r in range(2):
    for c in range(3):
        ax = fig.add_subplot(gs[r, 2 + c])
        ax.imshow(imgs[idx + 1])
        ax.set_title(titles[idx + 1], fontsize=25, pad=8)
        ax.text(0.5, -0.05, values[idx + 1], fontsize=18, ha='center', va='top', transform=ax.transAxes)
        ax.axis("off")
        idx += 1

fig.suptitle("Zoom-In Insets for x3 Super-Resolution Results(DIV2k-0282)", fontsize=30, fontweight='bold', y=1.1)
plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

rect = (700, 800, 400 ,300)
x, y, w, h = rect

img1 = Image.open("original/05.png")
img1 = zoom_inset(
    img1,
    rect = rect,
    scale = 2,
    corner = "top_right",
    rect_color = (0, 191, 255), 
    rect_thickness = 15,   
    inset_border_color = (0, 191, 255), 
    inset_border = 15,      
    margin = 10,  
    connector = False   
)
img2 = Image.open("hr_bicubic_x4/05_HR.png").convert("RGB").crop((x, y, x + w, y + h))
img3 = Image.open("srcnn_x4/05_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img4 = Image.open("fsrcnn_x4/05_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img5 = Image.open("espcn_x4/05_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img6 = Image.open("edsr_x4/05_SR.png").convert("RGB").crop((x, y, x + w, y + h))
img7 = Image.open("imdn_x4/05_SR.png").convert("RGB").crop((x, y, x + w, y + h))

imgs = [img1, img2, img3, img4, img5, img6, img7]
titles = ["Original", "Bicubic x4", "SRCNN x4", "FSRCNN x4", "ESPCN x4", "EDSR x4", "IMDN x4"]
values = [
    "(PSNR/SSIM)",
    "(28.49dB/0.8499)",
    "(29.10dB/0.8683)",
    "(29.31dB/0.8687)",
    "(29.19dB/0.8612)",
    "(30.40dB/0.8910)",
    "(30.12dB/0.8843)"
]

fig = plt.figure(figsize=(25, 8), constrained_layout=True)
gs = fig.add_gridspec(2, 5)

ax1 = fig.add_subplot(gs[:, :2])
ax1.imshow(imgs[0])
ax1.set_title(titles[0], fontsize=30, pad=15)
ax1.text(0.5, -0.03, values[0], fontsize=25, ha='center', va='top', transform=ax1.transAxes)
ax1.axis("off")

idx = 0
for r in range(2):
    for c in range(3):
        ax = fig.add_subplot(gs[r, 2 + c])
        ax.imshow(imgs[idx + 1])
        ax.set_title(titles[idx + 1], fontsize=25, pad=8)
        ax.text(0.5, -0.05, values[idx + 1], fontsize=18, ha='center', va='top', transform=ax.transAxes)
        ax.axis("off")
        idx += 1

fig.suptitle("Zoom-In Insets for x4 Super-Resolution Results(DIV2k-0540)", fontsize=30, fontweight='bold', y=1.1)
plt.show()
